In [29]:
import skimage
import os
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from skimage import data
from skimage import io
from sklearn.mixture import GaussianMixture
import glob
import keras
import tqdm
from skimage.viewer import ImageViewer
import cv2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import stop_words
from keras.preprocessing import image
from keras.models import Sequential
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras import utils
%matplotlib inline

In [30]:
#Define File-Path to Users Folder
folder_list = ['#foodporn',"#nightlife","#cosmetics","#rockclimbing"]
image_path = "/Users/kmotwani/Desktop/Me/Education/Courses/Capstone Project/Insta Images/"

In [35]:
#Helper Function to get images from path
def get_df(path, list_input, thresh):
    final_list = []
    for ind, i in enumerate(list_input):
        temp_path = path + i
        print(i,"\n")
        count = 0
        for j in glob.glob(temp_path + '/*.jpg'):
            temp_dict = {}
            file_name = j.replace(temp_path,'')[1:]
            img = image.load_img(j, target_size=(64, 64))
            try:
                with open(temp_path+"/"+file_name[:-4]+'.txt', encoding="utf-8") as f:
                    content = f.readlines()
                    caption = ' '.join([x.strip() for x in content])
            except FileNotFoundError:
                continue
            temp_dict['File'], temp_dict['Response'] = file_name, int(ind)
            temp_dict['Image'], temp_dict['Caption'] = np.array(img), caption
            final_list.append(temp_dict)
            count += 1
            if count==thresh:
                break
    return pd.DataFrame(final_list) 


#Get Images from User List and Path
df = get_df(image_path, folder_list, 1000)
print("Number of images loaded:", len(df))
display(df.head())

#foodporn 

#nightlife 

#cosmetics 

#rockclimbing 

Number of images loaded: 4000


,Caption,File,Image,Response
0,...Siempre quise ser grande para poder comerme...,2018-04-08_21-17-49_UTC.jpg,"[[[122, 114, 112], [123, 116, 123], [139, 153,...",0
1,Lonche de Panela By @mojito.restobar . . . #va...,2018-04-08_21-36-39_UTC.jpg,"[[[167, 160, 201], [165, 158, 199], [173, 160,...",0
2,Brunching with friends is the best way to spen...,2018-04-08_20-19-50_UTC.jpg,"[[[28, 26, 37], [160, 185, 207], [162, 183, 20...",0
3,Obsessed with their little stoves. Ordered the...,2018-02-02_15-27-17_UTC.jpg,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0
4,"Hoje para o jantar foram dois ovos, em cima de...",2018-04-08_21-36-29_UTC.jpg,"[[[151, 154, 147], [105, 91, 78], [118, 143, 1...",0


In [36]:
#Split train and test set
use_df = df.sample(frac=1).reset_index(drop=True)
np.random.seed(9001)
msk = np.random.rand(len(use_df)) < 0.7
total_data_train = use_df[msk]
total_data_test = use_df[~msk]

#Define X,Y
x_train = np.array([i for i in total_data_train['Image'].as_matrix()])
x_test = np.array([i for i in total_data_test['Image'].as_matrix()])
y_train = utils.to_categorical(total_data_train['Response'].as_matrix(), num_classes=4)
y_test = utils.to_categorical(total_data_test['Response'].as_matrix(), num_classes=4)

In [ ]:
#Helper function to create CNN Model for Image Classification
def createModel(shape, classes_count):
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(128, 128, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (5, 5), activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(classes_count, activation='softmax'))
    model.summary()
    return model

#Helper function to run model and save intermediate weights
def run_model(model, x_train, y_train, x_test, y_test, batch_size, epochs):
    model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(lr=1e-4),metrics=['accuracy'])
    filepath="Saved_Models/KRM_weights-{epoch:02d}-{val_acc:.2f}.hdf5"
    check = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', period = 10)
    callbacks_list = [check]
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, 
                       validation_data=(x_test, y_test), callbacks=callbacks_list)
    return model, history

In [ ]:
#Create and Run model
model = createModel((128,128,3), 4)
run_model(model, x_train, y_train, x_test, y_test, batch_size=512, epochs=2000)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 124, 124, 32)      2432      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 58, 58, 32)        25632     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 29, 29, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 25, 25, 64)        51264     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 9216)              0         
__________

In [ ]:
count = 0
for ind,i in enumerate(x_train):
    count+=1
    if count<10:
        plt.figure()
        plt.imshow(i)
        print(y_train[ind])
    else:
        break

In [33]:
from keras.models import load_model
best_model = load_model('Backup/KRM_weights-260-0.69.hdf5')
best_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_78 (Conv2D)           (None, 32, 32, 256)       3328      
_________________________________________________________________
max_pooling2d_73 (MaxPooling (None, 16, 16, 256)       0         
_________________________________________________________________
dropout_73 (Dropout)         (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 8, 8, 128)         131200    
_________________________________________________________________
max_pooling2d_74 (MaxPooling (None, 4, 4, 128)         0         
_________________________________________________________________
dropout_74 (Dropout)         (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 2, 2, 64)          32832     
__________

In [24]:
def get_feature_maps(model, layer_id, input_text):
    model_ = Model(inputs=[model.input], outputs=[model.layers[layer_id].output])
    return model_.predict(input_text)


from keras.preprocessing import image
import numpy as np
from keras.models import Model

img1 = image.load_img('pavlos.png', target_size=(64, 64))
img2 = image.load_img('pavlos2.png', target_size=(64, 64))
imgs = [np.array(img1), np.array(img2)]

final = []
for i in imgs:
    temp_map = list(get_feature_maps(best_model, 9, i.reshape(1,64,64,3))[0])
    print(temp_map)
    final.append(temp_map)

[0.0, 0.0, 0.0, 0.0, 398.24268, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 623.027, 0.0, 159.7953, 0.0, 0.0, 0.0, 236.2793, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 182.89426, 0.0, 0.0, 0.0, 1.1758604, 0.0, 50.645958, 0.0, 0.0, 0.0, 0.0, 65.4882, 0.0, 0.0, 0.0, 30.122917, 203.19316, 0.0, 0.0, 0.0, 133.13985, 0.0, 0.0, 129.2935, 134.26262, 0.0, 0.0, 0.0, 239.01886, 0.0, 0.0, 0.0, 0.0, 108.569984, 0.0, 72.54968, 0.0, 8.242872, 0.0]
[0.0, 0.0, 0.0, 0.0, 300.3895, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 541.22473, 0.0, 53.649834, 0.0, 0.0, 0.0, 200.36305, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 60.92927, 0.0, 0.0, 0.0, 0.0, 0.0, 64.76068, 0.0, 0.0, 0.0, 0.0, 26.611198, 0.0, 0.0, 0.0, 9.71535, 64.19645, 0.0, 0.0, 0.0, 10.272739, 0.0, 0.0, 103.01413, 73.572495, 0.0, 0.0, 0.0, 221.52542, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 47.562546, 0.0]


In [41]:
%%file Image_CNN.py

from keras.models import load_model
import numpy as np
from keras.models import Model

class Main:
    
    #Initialize with saved model and embeddings
    def __init__(self, path_model):
        self.model = load_model(path_model)
        
    #Helper function to create feature maps
    def get_feature_maps(self, layer_id, input_text):
        model_ = Model(inputs=[self.model.input], outputs=[self.model.layers[layer_id].output])
        return model_.predict(input_text)
        
    #Helper function to predict 
    def predict(self, x):
        final_list = []
        print("CNN Prediction in progress.\n")
        for ind, i in enumerate(x):
            print(ind,"/", len(x))
            temp_map = list(self.get_feature_maps(9, i.reshape(1,64,64,3))[0])
            final_list.append(temp_map)
        return final_list
    

    #Helper fucntion to combine DF to predictions
    def combine(self, df, prediction, label):
        new_cols = np.zeros((len(df),len(prediction[0])))
        for ind, i in enumerate(prediction):
            new_cols[ind,:] = i
        for i in range(len(prediction[0])):
            df[label+str(i+1)] = new_cols[:,i]
        return df

Writing Image_CNN.py


In [42]:
import Image_CNN

#Import Model and Embedding
obj = Image_CNN.Main('Backup/KRM_weights-260-0.69.hdf5')

In [44]:
#Create DataFrame
import pandas as pd
df = pd.DataFrame([[imgs[0],123,"www.google.co.in"],[imgs[1],123,"www.hotmail.co.in"]], columns=['Image', 'ID','URL'])
display(df.head())

,Image,ID,URL
0,"[[[222, 224, 230], [222, 224, 230], [222, 224,...",123,www.google.co.in
1,"[[[188, 188, 195], [191, 191, 198], [192, 192,...",123,www.hotmail.co.in


In [45]:
#Define Post
x = df['Image']

In [46]:
#Get Predictions
pred = obj.predict(x)
print(pred)

CNN Prediction in progress.

0 / 2
1 / 2
[[0.0, 0.0, 0.0, 0.0, 398.24268, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 623.027, 0.0, 159.7953, 0.0, 0.0, 0.0, 236.2793, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 182.89426, 0.0, 0.0, 0.0, 1.1758604, 0.0, 50.645958, 0.0, 0.0, 0.0, 0.0, 65.4882, 0.0, 0.0, 0.0, 30.122917, 203.19316, 0.0, 0.0, 0.0, 133.13985, 0.0, 0.0, 129.2935, 134.26262, 0.0, 0.0, 0.0, 239.01886, 0.0, 0.0, 0.0, 0.0, 108.569984, 0.0, 72.54968, 0.0, 8.242872, 0.0], [0.0, 0.0, 0.0, 0.0, 300.3895, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 541.22473, 0.0, 53.649834, 0.0, 0.0, 0.0, 200.36305, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 60.92927, 0.0, 0.0, 0.0, 0.0, 0.0, 64.76068, 0.0, 0.0, 0.0, 0.0, 26.611198, 0.0, 0.0, 0.0, 9.71535, 64.19645, 0.0, 0.0, 0.0, 10.272739, 0.0, 0.0, 103.01413, 73.572495, 0.0, 0.0, 0.0, 221.52542, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 47.562546, 0.0]]


In [47]:
#Combine Prediction
df = obj.combine(df, pred, "CNN_Feature_")
display(df.head())

,Image,ID,URL,CNN_Feature_1,CNN_Feature_2,CNN_Feature_3,CNN_Feature_4,CNN_Feature_5,CNN_Feature_6,CNN_Feature_7,...,CNN_Feature_55,CNN_Feature_56,CNN_Feature_57,CNN_Feature_58,CNN_Feature_59,CNN_Feature_60,CNN_Feature_61,CNN_Feature_62,CNN_Feature_63,CNN_Feature_64
0,"[[[222, 224, 230], [222, 224, 230], [222, 224,...",123,www.google.co.in,0.0,0.0,0.0,0.0,398.242676,0.0,0.0,...,0.0,0.0,0.0,0.0,108.569984,0.0,72.549683,0.0,8.242872,0.0
1,"[[[188, 188, 195], [191, 191, 198], [192, 192,...",123,www.hotmail.co.in,0.0,0.0,0.0,0.0,300.389496,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,47.562546,0.0
